NOTES FOR STANDARDIZATION
- Need functional/config interface into search space parameters
- Separate search spaces for ortho dims

In [1]:
import numpy as np
from ng_link import NgState, link_utils

import zarr_io
import coarse_registration

In [2]:
def _zyx_vector_to_3x4(zyx_vector: np.ndarray):
    output = np.zeros((3, 4))
    
    # Set identity
    output[0, 0] = 1
    output[1, 1] = 1
    output[2, 2] = 1

    # Set translation vector
    output[0, 3] = zyx_vector[2]
    output[1, 3] = zyx_vector[1]
    output[2, 3] = zyx_vector[0]
    return output

def convert_matrix_3x4_to_5x6(matrix_3x4: np.ndarray) -> np.ndarray:
    # Initalize
    matrix_5x6 = np.zeros((5, 6), np.float16)
    np.fill_diagonal(matrix_5x6, 1)

    # Swap Rows 0 and 2; Swap Colums 0 and 2
    patch = np.copy(matrix_3x4)
    patch[[0, 2], :] = patch[[2, 0], :]
    patch[:, [0, 2]] = patch[:, [2, 0]]

    # Place patch in bottom-right corner
    matrix_5x6[2:6, 2:7] = patch

    return matrix_5x6

def apply_deskewing(matrix_3x4: np.ndarray, theta: float = -45) -> np.ndarray:
    # Deskewing
    # X vector => XZ direction
    deskew_factor = np.tan(np.deg2rad(theta))
    deskew = np.array([[1, 0, 0], [0, 1, 0], [deskew_factor, 0, 1]])
    matrix_3x4 = deskew @ matrix_3x4

    return matrix_3x4

# Simply applies the same registration across channels, b/c coreg is extra effort for little gain
# Notice the same tile layout, cx, and cy will be applied to each of the nested lists of paths. 
def create_ng_link(tile_paths: list[list[str]], 
                   tile_layout: np.ndarray, 
                   cx: np.ndarray, 
                   cy: np.ndarray,
                   vox_sizes_xyz: np.ndarray, 
                   channels: list[int],
                   max_dr: int = 200,
                   opacity: float = 1.0,
                   blend: str = "default",
                   output_json_path: str = ".") -> None:
    
    # Pre-processing
    cx[np.isnan(cx)] = 0    
    cy[np.isnan(cy)] = 0

    # Determine tile translations
    tile_translations_zyx = np.zeros((tile_layout.shape[0], tile_layout.shape[1], 3))
    for xi in range(tile_layout.shape[0]):
        for yi in range(tile_layout.shape[1]):
            if xi == 0 and yi == 0:  # TL corner tile is fixed
                continue
            
            if xi == 0: # Tile on top border 
                curr_h = cx[:, 0, xi, yi - 1]
                cum_h = tile_translations_zyx[xi, yi - 1]
                tile_translations_zyx[xi, yi] = curr_h + cum_h

            elif yi == 0: # Tile on left border
                curr_v = cy[:, 0, xi - 1, yi]
                cum_v = tile_translations_zyx[xi - 1, yi]
                tile_translations_zyx[xi, yi] = curr_v + cum_v

            else:
                curr_h = cx[:, 0, xi, yi - 1]
                curr_v = cy[:, 0, xi - 1, yi]
                cum_h = tile_translations_zyx[xi, yi - 1]
                cum_v = tile_translations_zyx[xi - 1, yi]
                tile_translations_zyx[xi, yi] = curr_h + cum_h + curr_v + cum_v

    # print('Net Coarse Registration')
    # print(tile_translations_zyx)

    tile_translations_zyx = tile_translations_zyx * 4   # FIXME: Convert offsets into original resolution, reversing downsampling
    
    # print('Scaled Net Coarse Registration')
    # print(tile_translations_zyx)

    # Generate input config
    layers = []  # Nueroglancer Tabs
    input_config = {
        "dimensions": {
            "x": {"voxel_size": vox_sizes_xyz[0], "unit": "microns"},
            "y": {"voxel_size": vox_sizes_xyz[1], "unit": "microns"},
            "z": {"voxel_size": vox_sizes_xyz[2], "unit": "microns"},
            "c'": {"voxel_size": 1, "unit": ""},
            "t": {"voxel_size": 0.001, "unit": "seconds"},
        },
        "layers": layers,
        "showScaleBar": False,
        "showAxisLines": False,
    }

    for channel_tile_paths, channel in zip(tile_paths, channels):
        hex_val: int = link_utils.wavelength_to_hex(channel)
        hex_str = f"#{str(hex(hex_val))[2:]}"

        sources = []  # Tiles within tabs
        layers.append(
            {
                "type": "image",  # Optional
                "source": sources,
                "channel": 0,  # Optional
                "shaderControls": {
                    "normalized": {"range": [0, max_dr]}
                },  # Optional  # Exaspim has low HDR
                "shader": {
                    "color": hex_str,
                    "emitter": "RGB",
                    "vec": "vec3",
                },
                "visible": True,  # Optional
                "opacity": opacity,
                "name": f"CH_{channel}",
                "blend": blend,
            }
        )

        for xi in range(tile_layout.shape[0]):
            for yi in range(tile_layout.shape[1]):
                tile_id = tile_layout[xi, yi]
                tr_zyx = tile_translations_zyx[xi, yi]
                
                url = f"s3://aind-open-data/{channel_tile_paths[tile_id]}"
                sources.append(
                    {"url": url, "transform_matrix": convert_matrix_3x4_to_5x6(apply_deskewing(_zyx_vector_to_3x4(tr_zyx))).tolist()}
                )
    
    # Generate the link
    neuroglancer_link = NgState(
        input_config=input_config,
        mount_service="s3",
        bucket_path="aind-open-data",
        output_json=output_json_path,
    )
    neuroglancer_link.save_state_as_json()
    # print(neuroglancer_link.get_url_link())

    return input_config

DATASETS: 
- Polina:
    - diSPIM_647403_2022-11-21_23-12-18 (DONE)
    - diSPIM_645736_2022-12-13_02-21-00 (DONE)
    - diSPIM_647403_2022-12-03_04-29-00 (Revisit)
    - diSPIM_645734_2022-11-22_12-31-48 (Revisit)

- Kevin:
    - diSPIM_657584_2023-03-23_09-24-11 (round1)
    - diSPIM_657584_2023-05-17_11-17-16 (round3)

In [3]:
# Polina's Stuff -- may want to copy/paste/modify this

bucket = 'aind-open-data'
dataset = 'diSPIM_645734_2022-11-22_12-31-48/diSPIM.zarr/'
downsampling_exp = 2

tile_layout = np.array([[9],
                        [8],
                        [7],
                        [6],
                        [5],
                        [4],
                        [3],
                        [2],
                        [1],
                        [0]])

vox_sizes_xyz = [0.298, 0.298, 0.176]  # In um
channels = [405, 488, 561, 638]
tile_volumes = []
tile_paths = []
for channel in channels:
    c_paths = []
    for i in range(tile_layout.shape[0]):
        if i < 10: 
            i = f"0{i}" 
        path = f"tile_X_00{i}_Y_0000_Z_0000_CH_0{channel}_cam1.zarr"
        c_paths.append(dataset + path)
        
        if channel == 405:   # Just selecting one
            tile = zarr_io.open_zarr_s3(bucket, dataset + path + f'/{downsampling_exp}')
            tile_volumes.append(tile.T[:,:,:,0,0])
    tile_paths.append(c_paths)

# Entirety of the registration
cx, cy = coarse_registration.compute_coarse_offsets(tile_layout, tile_volumes)


Top Id: 9, Bottom Id: 8
Top: (0, 0), Bot: (1, 0) [ 49. 275. -50.]
Top Id: 8, Bottom Id: 7
Top: (1, 0), Bot: (2, 0) [ 49. 275. -36.]
Top Id: 7, Bottom Id: 6
Top: (2, 0), Bot: (3, 0) [ 49. 275.  -5.]
Top Id: 6, Bottom Id: 5
Top: (3, 0), Bot: (4, 0) [  2. 286.   4.]
Top Id: 5, Bottom Id: 4
Top: (4, 0), Bot: (5, 0) [  1. 285.   3.]
Top Id: 4, Bottom Id: 3
Top: (5, 0), Bot: (6, 0) [ -1. 287.   2.]
Top Id: 3, Bottom Id: 2
Top: (6, 0), Bot: (7, 0) [ -1. 287.   1.]
Top Id: 2, Bottom Id: 1
Top: (7, 0), Bot: (8, 0) [  1. 275.   4.]
Top Id: 1, Bottom Id: 0
Top: (8, 0), Bot: (9, 0) [-15. 285.  -8.]


In [5]:
tile_volumes[0].shape   # Informs tweaking 

(576, 576, 2896)

In [4]:
# Entirety of the viewing 
create_ng_link(tile_paths, 
               tile_layout, 
               cx, 
               cy, 
               vox_sizes_xyz, 
               channels, 
               max_dr=400, 
               output_json_path='/home/jonathan.wong/sofima-testing')

{'dimensions': {'x': {'voxel_size': 0.298, 'unit': 'microns'},
  'y': {'voxel_size': 0.298, 'unit': 'microns'},
  'z': {'voxel_size': 0.176, 'unit': 'microns'},
  "c'": {'voxel_size': 1, 'unit': ''},
  't': {'voxel_size': 0.001, 'unit': 'seconds'}},
 'layers': [{'type': 'image',
   'source': [{'url': 'zarr://s3://aind-open-data/diSPIM_645734_2022-11-22_12-31-48/diSPIM.zarr/tile_X_0009_Y_0000_Z_0000_CH_0405_cam1.zarr',
     'transform': {'matrix': [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, 0.0, -1.0, 0.0],
       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 1.0, 0.0]],
      'outputDimensions': {'t': [0.001, 's'],
       "c'": [1, ''],
       'z': [1.76e-07, 'm'],
       'y': [2.98e-07, 'm'],
       'x': [2.98e-07, 'm']}}},
    {'url': 'zarr://s3://aind-open-data/diSPIM_645734_2022-11-22_12-31-48/diSPIM.zarr/tile_X_0008_Y_0000_Z_0000_CH_0405_cam1.zarr',
     'transform': {'matrix': [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      

In [ ]:
# For reference: 
# def create_ng_link(tile_paths: list[list[str]], 
#                    tile_layout: np.ndarray, 
#                    cx: np.ndarray, 
#                    cy: np.ndarray,
#                    vox_sizes_xyz: np.ndarray, 
#                    channels: list[int],
#                    max_dr: int = 200,
#                    opacity: float = 1.0,
#                    blend: str = "default",
#                    output_json_path: str = ".") -> None: